---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [7]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [8]:
def get_list_of_university_towns():
    
    df = pd.DataFrame([], columns=['State', 'RegionName'])

    with open('university_towns.txt', 'r') as file:

        state = ""

        for line in file:

            if '[edit]' in line:

                state = line[:line.find('[')].strip()

                continue

            region = line.strip()

            if '(' in region:

                region = region[:(region.find('(') - 1)]

            df = df.append(pd.DataFrame([[state, region]], columns=['State', 'RegionName']), ignore_index=True)

    return df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [9]:
def get_recession_start():
    df_GDP = pd.read_excel("gdplev.xls", skiprows = 7, names = ["Year_Quarter", "GDP"], parse_cols= "E, G")
    df_GDP = df_GDP.loc[212:]
    df_GDP['percGDP_change'] = df_GDP['GDP'].pct_change()
    df_GDP = df_GDP.reset_index(drop=True)
    df_GDP["decline"] = (df_GDP.percGDP_change < 0) & (df_GDP.percGDP_change.shift(periods = -1) < 0) 
    return df_GDP.iloc[df_GDP["decline"].idxmax()]["Year_Quarter"]
    
get_recession_start()

'2008q3'

In [10]:
def get_recession_end():
    df_GDP = pd.read_excel("gdplev.xls", skiprows = 7, names = ["Year_Quarter", "GDP"], parse_cols= "E, G")
    df_GDP = df_GDP.loc[246:]
    df_GDP['percGDP_change'] = df_GDP['GDP'].pct_change()
    df_GDP = df_GDP.reset_index(drop=True)
    df_GDP["Growth"] = (df_GDP.percGDP_change > 0) & (df_GDP.percGDP_change.shift(periods = -1) > 0) 
    return df_GDP.iloc[df_GDP["Growth"].idxmax()]["Year_Quarter"]
    
get_recession_end()

'2009q3'

In [11]:
def get_recession_bottom():
    df_GDP = pd.read_excel("gdplev.xls", skiprows = 7, names = ["Year_Quarter", "GDP"], parse_cols= "E, G")
    df_GDP = df_GDP.loc[212:]
    start_recession = get_recession_start()
    end_recession = get_recession_end()
    df_recessionlen = df_GDP.loc[df_GDP[df_GDP['Year_Quarter'] == start_recession].iloc[0].name : 
                                df_GDP[df_GDP['Year_Quarter'] == end_recession].iloc[0].name]
    
    recession_bottom = df_recessionlen[df_recessionlen.GDP == df_recessionlen['GDP'].min()]
    return recession_bottom.iloc[0][0]

get_recession_bottom() 

'2009q2'

In [16]:
def convert_housing_data_to_quarters():
    df_housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    df_housing['State'] = df_housing['State'].replace(states)
    df_housing.drop(['Metro','CountyName','RegionID','SizeRank'],axis = 1,inplace = 1)
    df_housing.set_index(['State', 'RegionName'], inplace=True)
    df_housing.drop(list(df_housing.columns)[:45], axis = 1, inplace = 1)
    housing_col = list(df_housing.columns)
    
    def new_col():
        years = list(range(2000,2017))
        quarters = ['q1','q2','q3','q4']
        quar_years = []
        for i in years:
            for y in quarters:
                quar_years.append((str(i)+y))
        return quar_years[:67]    

    Quart = [list(df_housing.columns)[u:u+3] for u in range(0, len(list(df_housing.columns)), 3)]


    return Quart
    
convert_housing_data_to_quarters()

[['2000-01', '2000-02', '2000-03'],
 ['2000-04', '2000-05', '2000-06'],
 ['2000-07', '2000-08', '2000-09'],
 ['2000-10', '2000-11', '2000-12'],
 ['2001-01', '2001-02', '2001-03'],
 ['2001-04', '2001-05', '2001-06'],
 ['2001-07', '2001-08', '2001-09'],
 ['2001-10', '2001-11', '2001-12'],
 ['2002-01', '2002-02', '2002-03'],
 ['2002-04', '2002-05', '2002-06'],
 ['2002-07', '2002-08', '2002-09'],
 ['2002-10', '2002-11', '2002-12'],
 ['2003-01', '2003-02', '2003-03'],
 ['2003-04', '2003-05', '2003-06'],
 ['2003-07', '2003-08', '2003-09'],
 ['2003-10', '2003-11', '2003-12'],
 ['2004-01', '2004-02', '2004-03'],
 ['2004-04', '2004-05', '2004-06'],
 ['2004-07', '2004-08', '2004-09'],
 ['2004-10', '2004-11', '2004-12'],
 ['2005-01', '2005-02', '2005-03'],
 ['2005-04', '2005-05', '2005-06'],
 ['2005-07', '2005-08', '2005-09'],
 ['2005-10', '2005-11', '2005-12'],
 ['2006-01', '2006-02', '2006-03'],
 ['2006-04', '2006-05', '2006-06'],
 ['2006-07', '2006-08', '2006-09'],
 ['2006-10', '2006-11', '200

In [41]:
from scipy import stats

def run_ttest():
    housing_q = convert_housing_data_to_quarters()
    begin = housing_q.columns.get_loc(get_recession_start())
    end = housing_q.columns.get_loc(get_recession_bottom())
    housing_q['Ratio'] = housing_q.iloc[:, begin - 1] / housing_q.iloc[:, end]
    housing_q = pd.DataFrame(housing_q.loc[:, 'Ratio'])
    uni_towns = get_list_of_university_towns()
    uni_towns = uni_towns.set_index(['State', 'RegionName'])
    univ_price= pd.merge(housing_q, uni_towns, how="inner", left_index=True, right_index=True)
    nonuniv = pd.merge(housing_q, uni_towns, how="outer", left_index=True, right_index=True, indicator=True)
    nonuniv = nonuniv.drop('_merge', axis=1)
    univ_price = univ_price.dropna()
    nonuniv = nonuniv.dropna()

    t, p = stats.ttest_ind(univ_price['Ratio'], nonuniv['Ratio'])

    t2, p2 = stats.ttest_ind(nonuniv['Ratio'], univ_price['Ratio'])

    result = True, p, "university town"

    return result
    

run_ttest()






(True, 0.34596775869784346, 'university town')